<a href="https://colab.research.google.com/github/nuckleheadninja/Coastal_BLUETech-Syndicate/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this is for vit vs imaagenet
import torch
import torch.nn as nn
import timm


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("surajit651/souvikdataset")

print("Path to dataset files:", path)

100%|██████████| 1.58G/1.58G [00:16<00:00, 105MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/surajit651/souvikdataset/versions/1


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adidev001/procesed-again-costal-polutant")

print("Path to dataset files:", path)

100%|██████████| 338M/338M [00:02<00:00, 168MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/adidev001/procesed-again-costal-polutant/versions/2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
datasetPath = "/root/.cache/kagglehub/datasets/adidev001/procesed-again-costal-polutant/versions/2/balanced_dataset"


In [ ]:
from torchvision import datasets

dataset = datasets.ImageFolder(datasetPath)
print(dataset.classes)


['cardboard', 'clean_water', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']


In [ ]:
!pip install timm
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm


In [ ]:
def prepareTransforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])


In [ ]:
def loadDataset(path):
    transform = prepareTransforms()
    dataset = datasets.ImageFolder(path, transform=transform)
    return dataset
dataset = loadDataset(datasetPath)
print("Total images:", len(dataset))
print("Classes:", dataset.classes)


Total images: 14493
Classes: ['cardboard', 'clean_water', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']


In [ ]:
from torch.utils.data import random_split

def splitDataset(dataset):
    trainSize = int(0.7 * len(dataset))
    valSize = len(dataset) - trainSize

    trainData, valData = random_split(dataset, [trainSize, valSize])



    print("Training images:", len(trainData))
    print("Validation images:", len(valData))

    return trainData, valData
trainData, valData = splitDataset(dataset)



Training images: 10145
Validation images: 4348


In [ ]:
def createLoaders(trainData, valData):
    trainLoader = DataLoader(trainData, batch_size=32, shuffle=True)
    valLoader = DataLoader(valData, batch_size=32, shuffle=False)
    return trainLoader, valLoader
trainLoader, valLoader = createLoaders(trainData, valData)


In [ ]:
def loadEfficientNet(classCount):
    model = timm.create_model(
        "efficientnet_b0",
        pretrained=True,
        num_classes=classCount
    )
    return model
classCount = len(dataset.classes)
model = loadEfficientNet(classCount)


In [ ]:
def freezeModel(model):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier.parameters():
        param.requires_grad = True
freezeModel(model)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def trainModel(model, loader):
    model.train()
    totalLoss = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = lossFunction(outputs, labels)
        loss.backward()
        optimizer.step()

        totalLoss += loss.item()

    return totalLoss / len(loader)
def validateModel(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return (correct / total) * 100
for epoch in range(5):
    trainLoss = trainModel(model, trainLoader)
    valAccuracy = validateModel(model, valLoader)

    print(f"Epoch {epoch+1}")
    print("Train Loss:", trainLoss)
    print("Val Accuracy:", valAccuracy)
torch.save(model.state_dict(), "efficientnetMarine.pth")


Epoch 1
Train Loss: 1.5557196124357247
Val Accuracy: 73.62005519779208
Epoch 2
Train Loss: 0.6639168580185693
Val Accuracy: 82.15271389144434
Epoch 3
Train Loss: 0.5042528400443634
Val Accuracy: 83.90064397424104
Epoch 4
Train Loss: 0.4395507459511172
Val Accuracy: 86.17755289788408
Epoch 5
Train Loss: 0.39042659501000393
Val Accuracy: 86.86752529898804


In [ ]:
# ==============================
# 1. INSTALL & IMPORT LIBRARIES
# ==============================
!pip install timm tqdm

import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm
from tqdm import tqdm

# ==============================
# 2. DATASET PATH (CHANGE IF NEEDED)
# ==============================
datasetPath = "/root/.cache/kagglehub/datasets/adidev001/procesed-again-costal-polutant/versions/2/balanced_dataset"

# ==============================
# 3. IMAGE TRANSFORMS
# ==============================
def prepareTransforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

# ==============================
# 4. LOAD DATASET
# ==============================
dataset = datasets.ImageFolder(datasetPath, transform=prepareTransforms())
print("Total images:", len(dataset))
print("Classes:", dataset.classes)

# ==============================
# 5. TRAIN / VAL SPLIT
# ==============================
def splitDataset(dataset):
    trainSize = int(0.7 * len(dataset))
    valSize = len(dataset) - trainSize
    trainData, valData = random_split(dataset, [trainSize, valSize])

    print("Training images:", len(trainData))
    print("Validation images:", len(valData))

    return trainData, valData

trainData, valData = splitDataset(dataset)

# ==============================
# 6. DATALOADERS
# ==============================
trainLoader = DataLoader(trainData, batch_size=32, shuffle=True)
valLoader = DataLoader(valData, batch_size=32, shuffle=False)

# ==============================
# 7. LOAD PRETRAINED EFFICIENTNET
# ==============================
classCount = len(dataset.classes)

model = timm.create_model(
    "efficientnet_b0",
    pretrained=True,
    num_classes=classCount
)

# ==============================
# 8. UNFREEZE ENTIRE MODEL
# ==============================
for param in model.parameters():
    param.requires_grad = True

# ==============================
# 9. DEVICE, LOSS, OPTIMIZER
# ==============================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# ==============================
# 10. TRAIN FUNCTION (WITH PROGRESS)
# ==============================
def trainModel(model, loader, epochNumber):
    model.train()
    totalLoss = 0

    progressBar = tqdm(loader, desc=f"Training Epoch {epochNumber}", leave=False)

    for images, labels in progressBar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = lossFunction(outputs, labels)
        loss.backward()
        optimizer.step()

        totalLoss += loss.item()
        progressBar.set_postfix(loss=loss.item())

    return totalLoss / len(loader)

# ==============================
# 11. VALIDATION FUNCTION
# ==============================
def validateModel(model, loader, epochNumber):
    model.eval()
    correct = 0
    total = 0

    progressBar = tqdm(loader, desc=f"Validating Epoch {epochNumber}", leave=False)

    with torch.no_grad():
        for images, labels in progressBar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return (correct / total) * 100

# ==============================
# 12. TRAIN LOOP (FULL FINE-TUNING)
# ==============================
epochs = 9
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    trainLoss = trainModel(model, trainLoader, epoch+1)
    valAccuracy = validateModel(model, valLoader, epoch+1)

    print("Train Loss:", trainLoss)
    print("Val Accuracy:", valAccuracy)

# ==============================
# 13. SAVE MODEL
# ==============================
print("Model saved successfully")
torch.save(model.state_dict(), "efficientnetMarineBest90.pth")


Total images: 14493
Classes: ['cardboard', 'clean_water', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']
Training images: 10145
Validation images: 4348

Epoch 1/9


Train Loss: 2.058082681969277
Val Accuracy: 67.38730450781969

Epoch 2/9


Train Loss: 0.8060232837620022
Val Accuracy: 79.18583256669733

Epoch 3/9


Train Loss: 0.5070470715843657
Val Accuracy: 83.76264949402024

Epoch 4/9


Train Loss: 0.4034265555300803
Val Accuracy: 86.68353265869365

Epoch 5/9


Train Loss: 0.30667541300257045
Val Accuracy: 88.43146274149034

Epoch 6/9


Train Loss: 0.24689928263004096
Val Accuracy: 90.04139834406624

Epoch 7/9


Train Loss: 0.21499980535686203
Val Accuracy: 90.43238270469182

Epoch 8/9


Train Loss: 0.17936713235312193
Val Accuracy: 90.91536338546457

Epoch 9/9


Train Loss: 0.14317669420731519
Val Accuracy: 92.0883164673413
Model saved successfully


In [50]:
!pip install git+https://github.com/openai/CLIP.git
import clip
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

def loadClipModel():
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess

clipModel, clipPreprocess = loadClipModel()
['cardboard', 'clean_water', 'glass',
 'marine_trash', 'metal', 'oil_spill',
 'paper', 'plastic']
clipSeverityLabels = [
    "large scale marine pollution with many plastic wastes and trash piles",
    "heavy ocean pollution with dense garbage floating on water",
    "massive oil spill covering the sea surface",
    "clean ocean water with no visible pollution",
    "natural beach or sea with no trash"
]
HEAVY_INDICES = [0, 1, 2]     # heavy pollution
GENERAL_SEVERITY_THRESHOLD = 0.35
OIL_SPILL_INDEX = 2
OIL_SEVERITY_THRESHOLD = 0.25
def clipSeverityCheck(imagePath):
    image = clipPreprocess(Image.open(imagePath)).unsqueeze(0).to(device)
    text = clip.tokenize(clipSeverityLabels).to(device)

    with torch.no_grad():
        imageFeatures = clipModel.encode_image(image)
        textFeatures = clipModel.encode_text(text)
        similarity = (imageFeatures @ textFeatures.T).softmax(dim=-1)

    scores = similarity.squeeze().cpu().numpy()
    bestIndex = scores.argmax()
    confidence = scores[bestIndex]

    return bestIndex, confidence
def finalPrediction(imagePath):
    severityIndex, severityConf = clipSeverityCheck(imagePath)

    # ---- Oil spill special handling
    if severityIndex == OIL_SPILL_INDEX and severityConf >= OIL_SEVERITY_THRESHOLD:
        cnnScores = cnnPredict(imagePath)
        bestIndex = cnnScores.argmax()
        return dataset.classes[bestIndex], cnnScores[bestIndex] * 100

    # ---- General rejection
    if severityIndex not in HEAVY_INDICES or severityConf < GENERAL_SEVERITY_THRESHOLD:
        return "No marine pollution detected", severityConf * 100

    # ---- Heavy pollution → CNN classify
    cnnScores = cnnPredict(imagePath)
    bestIndex = cnnScores.argmax()
    return dataset.classes[bestIndex], cnnScores[bestIndex] * 100
def predictImage(imagePath):
    print("Analyzing image:", imagePath)

    severityIndex, severityConf = clipSeverityCheck(imagePath)

    print("\nCLIP Severity Analysis")
    print("----------------------")
    print("Matched description:", clipSeverityLabels[severityIndex])
    print(f"Confidence: {severityConf*100:.2f}%")

    label, confidence = finalPrediction(imagePath)

    print("\nFINAL RESULT")
    print("------------")
    print("Prediction:", label)
    print(f"Confidence: {confidence:.2f}%")
testImagePath = "/content/WhatsApp Image 2026-02-07 at 9.55.51 PM.jpeg"
predictImage(testImagePath)



  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2i277u2b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2i277u2b
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Analyzing image: /content/WhatsApp Image 2026-02-07 at 9.55.51 PM.jpeg

CLIP Severity Analysis
----------------------
Matched description: clean ocean water with no visible pollution
Confidence: 86.94%

FINAL RESULT
------------
Prediction: No marine pollution detected
Confidence: 86.94%


In [ ]:
def clipSeverityCheck(imagePath):
    image = clipPreprocess(Image.open(imagePath)).unsqueeze(0).to(device)
    text = clip.tokenize(clipSeverityLabels).to(device)

    with torch.no_grad():
        imageFeatures = clipModel.encode_image(image)
        textFeatures = clipModel.encode_text(text)

        similarity = (imageFeatures @ textFeatures.T).softmax(dim=-1)

    scores = similarity.squeeze().cpu().numpy()
    bestIndex = scores.argmax()
    confidence = scores[bestIndex]

    return bestIndex, confidence
HEAVY_INDICES = [0, 1, 2]  # first three labels are heavy pollution
SEVERITY_THRESHOLD = 0.35
def finalPrediction(imagePath):
    severityIndex, severityConf = clipSeverityCheck(imagePath)

    # --------------------------------
    # NOT HEAVY POLLUTION → REJECT
    # --------------------------------
    if severityIndex not in HEAVY_INDICES or severityConf < SEVERITY_THRESHOLD:
        return "No marine pollution detected", severityConf * 100

    # --------------------------------
    # HEAVY POLLUTION → CLASSIFY TYPE
    # --------------------------------
    cnnScores = cnnPredict(imagePath)
    bestIndex = cnnScores.argmax()
    confidence = cnnScores[bestIndex] * 100

    return dataset.classes[bestIndex], confidence
def predictImage(imagePath):
    print("Analyzing image...")
    print("Image path:", imagePath)

    # CLIP severity check
    severityIndex, severityConf = clipSeverityCheck(imagePath)

    print("\nCLIP Severity Analysis")
    print("----------------------")
    print("Matched description:", clipSeverityLabels[severityIndex])
    print(f"Confidence: {severityConf * 100:.2f}%")

    # Decision gate
    if severityIndex not in HEAVY_INDICES or severityConf < SEVERITY_THRESHOLD:
        print("\nFINAL RESULT")
        print("------------")
        print("No marine pollution detected")
        return

    # CNN classification
    cnnScores = cnnPredict(imagePath)
    bestIndex = cnnScores.argmax()
    confidence = cnnScores[bestIndex] * 100

    print("\nCNN Classification")
    print("------------------")
    print("Pollution type:", dataset.classes[bestIndex])
    print(f"Confidence: {confidence:.2f}%")

    print("\nFINAL RESULT")
    print("------------")
    print(f"Heavy marine pollution detected → {dataset.classes[bestIndex]}")
testImagePath = "/content/pexels-mika-photogenius-641920605-30379620.jpg"  # change this path
predictImage(testImagePath)



Analyzing image...
Image path: /content/pexels-mika-photogenius-641920605-30379620.jpg

CLIP Severity Analysis
----------------------
Matched description: clean ocean water with no visible pollution
Confidence: 47.25%

FINAL RESULT
------------
No marine pollution detected


In [ ]:
def predictImage(imagePath):
    print("Analyzing image...")
    print("Image path:", imagePath)

    # CLIP severity check
    severityIndex, severityConf = clipSeverityCheck(imagePath)

    print("\nCLIP Severity Analysis")
    print("----------------------")
    print("Matched description:", clipSeverityLabels[severityIndex])
    print(f"Confidence: {severityConf * 100:.2f}%")

    # Decision gate
    if severityIndex not in HEAVY_INDICES or severityConf < SEVERITY_THRESHOLD:
        print("\nFINAL RESULT")
        print("------------")
        print("No marine pollution detected")
        return

    # CNN classification
    cnnScores = cnnPredict(imagePath)
    bestIndex = cnnScores.argmax()
    confidence = cnnScores[bestIndex] * 100

    print("\nCNN Classification")
    print("------------------")
    print("Pollution type:", dataset.classes[bestIndex])
    print(f"Confidence: {confidence:.2f}%")

    print("\nFINAL RESULT")
    print("------------")
    print(f"Heavy marine pollution detected → {dataset.classes[bestIndex]}")
testImagePath = "/content/oilspill.jpg"  # change this path
predictImage(testImagePath)



Analyzing image...
Image path: /content/oilspill.jpg


NameError: name 'clipSeverityCheck' is not defined

In [ ]:
def clipPredict(imagePath):
    image = clipPreprocess(Image.open(imagePath)).unsqueeze(0).to(device)
    text = clip.tokenize(clipLabels).to(device)

    with torch.no_grad():
        imageFeatures = clipModel.encode_image(image)
        textFeatures = clipModel.encode_text(text)

        similarities = (imageFeatures @ textFeatures.T).softmax(dim=-1)

    return similarities.squeeze().cpu().numpy()
def cnnPredict(imagePath):
    image = Image.open(imagePath)
    image = prepareTransforms()(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)

    return probabilities.squeeze().cpu().numpy()
def ensemblePredict(imagePath):
    cnnScores = cnnPredict(imagePath)
    clipScores = clipPredict(imagePath)

    finalScores = (cnnScores * 0.6) + (clipScores * 0.4)

    bestIndex = finalScores.argmax()
    confidence = finalScores[bestIndex] * 100

    return dataset.classes[bestIndex], confidence
label, confidence = ensemblePredict("/content/oilspill.jpg")
print("Prediction:", label)
print("Confidence:", confidence)


Prediction: oil_spill
Confidence: 68.92578
